In [1]:
import menpo.io as mio
import os
from sklearn import svm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import Models
from Models import ChangeVector, CenterPointLandmark
path_to_svm_training_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'


def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

#process changingVector, reduce dimension from 2x68 to 1x68, by process PCA
def pca(changeVector):
    X = np.array(changeVector)
    pca_model = PCA(n_components=1)
    return pca_model.fit_transform(X)


training_images = mio.import_images(path_to_svm_training_database, verbose=True)
training_images = training_images.map(process)

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'


Found 1140 assets, index the returned LazyList to import.


In [41]:
import math
def landmark_normalize(landmark):
    vector_max = 0
    for p in landmark:
        if(math.sqrt(float(p[0])*float(p[0]) + float(p[1])*float(p[1])) > vector_max):
            vector_max = math.sqrt(p[0]*p[0] + p[1]*p[1])
    for p in landmark:
        p[0] = p[0]/float(vector_max)
        p[1] = p[1]/float(vector_max)
    return landmark

In [37]:
def process_center_point_landmark(landmark):
    x_array = []
    y_array = []
    xmean = 0
    ymeam = 0
    for p in landmark:
        x_array.append(p[0])
        y_array.append(p[1])
#     return x_array, y_array
    xmean = np.mean(x_array)
    ymean = np.mean(y_array)
    center_point_landmark = []
    for i in range(0,68):
        center_point_landmark.append([x_array[i] - float(xmean), y_array[i] - float(ymean)])
    return landmark_normalize(center_point_landmark)
#     return x_array, y_array

68

In [42]:
#create training data
from Models import CenterPointLandmark

count = 0;
svm_training_data = []
while(count < len(training_images)):
    file_path = str(training_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            if(int(float(x)) not in data_facs and int(float(x))!= 0):
                data_facs.append(int(float(x)))
    data_facs.sort()
    fi.close()
    
    centerPointLandmark = []
#     landmark_neutral = process_center_point_landmark(training_images[count].landmarks['PTS'].lms.points)
    landmark_perk = process_center_point_landmark(training_images[count + 1].landmarks['PTS'].lms.points)

#     svm_training_data.append(CenterPointLandmark([], landmark_neutral, gt_emotion))
    svm_training_data.append(CenterPointLandmark(data_facs, landmark_perk, gt_emotion))
    count = count + 2
#     centerPointVector = []

In [48]:
svm_training_data[1].facs

[1, 2, 20, 21, 25]

In [112]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor

facs = []
#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if(int(facs_code) not in facs and int(facs_code)!= 0):
            facs.append(facs_code)
facs.sort()

#create model for each action unit in facs[]
models = []
au_models_score = []
# SVC
# for au in facs:
#     x_training = []
#     y_label = []
#     #create label array
#     for data in svm_training_data:
#         if(au in data.facs):
#             y_label.append(1)
#         else:
#             y_label.append(0)
#         #create training data: 1x68 array, result of PCA process
#         vector = []
#         for tmp in data.landmarkChange:
#             vector.append(tmp[0])
#             vector.append(tmp[1])
#         x_training.append(vector)
#     x_training_normalized = normalize(x_training, norm='max')
#     clf = svm.LinearSVC()
#     clf.fit(x_training_normalized, y_label)
#     au_models_score.append(clf.score(x_training, y_label))
#     models.append(clf)

#Random forest
x_training = []
y_label = []
#create label array
for data in svm_training_data:
#     y_tmp.append(data.facs)
    #create training data: 1x68 array, result of PCA process
    vector = []
#     y_tmp = [0 for x in range(max(facs) +1)]
#     if(len(data.facs) > 0):
#         for f in data.facs:
#             y_tmp[f] = 1
    y_tmp = [0 for x in range(20)]
    y_tmp[int(data.emotion_label)] = 1
    y_label.append(y_tmp)
        
    for tmp in data.landmarkChange:
        vector.append(tmp[0])
        vector.append(tmp[1])
    x_training.append(vector)
x_training_normalized = normalize(x_training, norm='max')


In [107]:
data.emotion_label

'010'

In [113]:
clf = RandomForestClassifier(100, random_state = 0)
# clf = svm.LinearSVC()
model = MultiOutputRegressor(clf)
model.fit(x_training_normalized, y_label)
au_models_score.append(model.score(x_training, y_label))

print(au_models_score)

#create testing data
svm_testing_data = []
path_to_svm_testing_database = "/Programing/GR/Code/CK+/test-aam-images/**/**/**/*"
testing_images = mio.import_images(path_to_svm_testing_database, verbose=True)
testing_images = testing_images.map(process)

count = 0;
while(count < len(testing_images)):
    file_path = str(testing_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            if(int(float(x)) not in data_facs and int(float(x)) != 0):
                data_facs.append(int(float(x)))
    #print(array)
    fi.close()
    
    centerPointLandmark = []
#     landmark_neutral = process_center_point_landmark(testing_images[count].landmarks['PTS'].lms.points)
    landmark_perk = process_center_point_landmark(testing_images[count + 1].landmarks['PTS'].lms.points)

#     svm_testing_data.append(CenterPointLandmark([], landmark_neutral, gt_emotion))
    svm_testing_data.append(CenterPointLandmark(data_facs, landmark_perk, gt_emotion))
    count = count + 2

    
#evaluate trained model with test data and get score
print('#######')
print('Score: ')
# SVC score
# for au in facs:
#     x_training = []
#     y_label = []
#     #create label array
#     for data in svm_testing_data:
#         if(au in data.facs):
#             y_label.append(1)
#         else:
#             y_label.append(0)
#         #create training data: 1x68 array, result of PCA process
#         vector = []
#         for tmp in data.landmarkChange:
#             vector.append(tmp[0])
#             vector.append(tmp[1])
#         x_training.append(vector)
#     print(models[facs.index(au)].score(x_training, y_label))
    
# Random Forest score
x_training = []
y_label = []
#create label array
for data in svm_testing_data:
#     y_tmp = [0 for x in range(max(facs) +1)]
#     if(len(data.facs) > 0):
#         for f in data.facs:
#             y_tmp[f] = 1
    y_tmp = [0 for x in range(20)]
    y_tmp[int(data.emotion_label)] = 1
    y_label.append(y_tmp)
    #create training data: 1x68 array, result of PCA process
    vector = []
    for tmp in data.landmarkChange:
        vector.append(tmp[0])
        vector.append(tmp[1])
    x_training.append(vector)
print(model.score(x_training, y_label))


[0.4801817407945572]
Found 46 assets, index the returned LazyList to import.
#######
Score: 
0.5797143259217562


In [46]:
normalize([x_training[1]], norm='max')

array([[-0.5399615 , -0.90301431, -0.29876248, -0.90150064, -0.0620954 ,
        -0.87703413,  0.17019436, -0.81914416,  0.39685127, -0.74220763,
         0.60260231, -0.62224664,  0.7706943 , -0.45134974,  0.90658033,
        -0.25673187,  0.92967959, -0.02457927,  0.9035998 ,  0.23118287,
         0.80127929,  0.46811442,  0.63515176,  0.66385767,  0.4302579 ,
         0.82048945,  0.19294654,  0.91614754, -0.05986159,  0.97130905,
        -0.31559602,  0.99975543, -0.57212124,  1.        , -0.58666985,
        -0.78926123, -0.70134957, -0.64635451, -0.73031159, -0.4644722 ,
        -0.70005212, -0.28071379, -0.65013761, -0.10309227, -0.68757359,
         0.10857452, -0.70004397,  0.29216899, -0.75147976,  0.4692407 ,
        -0.73847537,  0.65086323, -0.63949463,  0.80382369, -0.51710168,
        -0.00463808, -0.35501027, -0.01708243, -0.19402578, -0.02822695,
        -0.03330257, -0.03210684,  0.13736285, -0.26085433,  0.14806096,
        -0.1399006 ,  0.15106891, -0.02032761,  0.1

In [47]:
wrong_predict = 0
au_score = []
for data in svm_testing_data:
    print('#####')
    local_wrong_predict = 0
    local_accurate_predict = 0
    tmp = []
    predict = []
    
    for vector in data.landmarkChange:
        tmp.append(vector[0])
        tmp.append(vector[1])
        
    for model in models:
        if(model.predict([tmp]) >= 0.5):
            predict.append([facs[models.index(model)], model.predict([tmp])[0]])
            #print(facs[models.index(model)])
            if(facs[models.index(model)] not in data.facs):
                local_wrong_predict += 1
            else: 
                local_accurate_predict += 1
        else:
            if(facs[models.index(model)] in data.facs):
                local_wrong_predict += 1
    print(predict)
    print(local_accurate_predict)
    if(len(data.facs) != 0):
        au_score.append(float(local_accurate_predict)/float(len(data.facs)))
    print("---")
    data.facs.sort()
    for gt_facs in data.facs:
        print([gt_facs, models[facs.index(gt_facs)].predict([tmp])[0]])
    wrong_predict += local_wrong_predict
    
print(wrong_predict)
print(sum(au_score)/float(len(au_score)))

#####
[[1, 1], [2, 1], [15, 1], [17, 1]]
4
---
[1, 1]
[2, 1]
[3, 0]
[4, 0]
[15, 1]
[17, 1]
[23, 0]
#####
[[2, 1], [4, 1], [17, 1]]
2
---
[1, 0]
[3, 0]
[4, 1]
[5, 0]
[15, 0]
[17, 1]
[23, 0]
#####
[[1, 1], [2, 1], [25, 1]]
3
---
[1, 1]
[2, 1]
[3, 0]
[4, 0]
[5, 0]
[14, 0]
[17, 0]
[20, 0]
[25, 1]
[26, 0]
#####
[[1, 1], [2, 1], [4, 1], [15, 1], [17, 1]]
5
---
[1, 1]
[2, 1]
[3, 0]
[4, 1]
[15, 1]
[17, 1]
#####
[[2, 1], [4, 1], [17, 1]]
3
---
[2, 1]
[3, 0]
[4, 1]
[5, 0]
[17, 1]
[23, 0]
[38, 0]
#####
[[2, 1], [4, 1]]
0
---
[14, 0]
#####
[[1, 1], [2, 1], [4, 1], [25, 1]]
4
---
[1, 1]
[2, 1]
[3, 0]
[4, 1]
[5, 0]
[14, 0]
[16, 0]
[20, 0]
[25, 1]
[38, 0]
#####
[[4, 1], [7, 1], [17, 1], [23, 1], [24, 1]]
3
---
[2, 0]
[4, 1]
[5, 0]
[17, 1]
[23, 1]
[38, 0]
#####
[[4, 1], [17, 1]]
0
---
[14, 0]
#####
[[1, 1], [2, 1], [4, 1], [15, 1], [17, 1]]
4
---
[1, 1]
[3, 0]
[4, 1]
[15, 1]
[17, 1]
#####
[[1, 1], [2, 1], [4, 1], [17, 1]]
3
---
[2, 1]
[3, 0]
[4, 1]
[5, 0]
[9, 0]
[15, 0]
[17, 1]
[23, 0]
[30, 0]
#####
[